In [28]:
import pandas as pd
import networkx as nx
from math import exp
import matplotlib.pyplot as plt
import numpy as np

nodes = pd.read_csv('routes.csv', header=None, names=['i', 'j'])
g = nx.from_pandas_edgelist(nodes, 'i', 'j')
print('Number of nodes', g.number_of_nodes())
print('Number of edges', g.number_of_edges())

# Initialisation

# We have a dataframe of countries (nodes)
countries = pd.read_csv('data.csv')
countries.set_index('country', inplace=True)

# For each country, we need the travellers going from the neighboring countries
def return_list_neighbors(i, g):
    return [j for j in g.neighbors(i)]
print('Albania\'s neighbors :', return_list_neighbors('Albania', g))
def compute_sum_travelers_neighboring_countries(g):
    # check what this function returns (format: index or content)
    vi = []
    for i in countries.index:
        neighbors = return_list_neighbors(i, g)
        sum = 0
        for n in neighbors:
            sum += countries.loc[i]['travellers']
        vi.append(sum/1000000)
    return vi
countries['vi'] = compute_sum_travelers_neighboring_countries(g)

# For t=0 :
countries['incubation0'] = np.zeros(countries.shape[0])
countries['sick0'] = np.zeros(countries.shape[0])
countries['recovered0'] = np.zeros(countries.shape[0])
countries['infected0'] = np.zeros(countries.shape[0])
countries['healthy0'] = countries['population']

# W_ij the weight between country i and j represents the number of travellers between i and j
def compute_edge_weight(i, j):
    return countries.loc[i]['travellers']*countries.loc[j]['travellers']/(countries.loc[i]['vi']*10**12)
        
def add_edge_weights():
    for e in g.edges():
        g[e[0]][e[1]]['weight'] = compute_edge_weight(e[0], e[1])

#nx.draw(g, pos=nx.spring_layout(g), node_color='b', node_size=50, with_labels=True)
#nx.draw_networkx_nodes(g, pos=nx.spring_layout(g), node_color='b', node_size=50, with_labels=True)
#nx.draw_networkx_labels(g, pos=nx.spring_layout(g))
#plt.show()


# Our model :
r0 = 3
alpha = 2/5
end = 10
mortality_rate = 0.2

countries.loc['France']['infected0'] = 0.0001
countries.loc['France']['incubation0'] = 0.0001
countries.loc['France']['healthy0'] = countries.loc['France']['population'] - countries.loc['France']['incubation0']
countries.loc['United States']['infected0'] = 0.0001
countries.loc['United States']['incubation0'] = 0.0001
countries.loc['United States']['healthy0'] = countries.loc['United States']['population'] - countries.loc['United States']['incubation0']

def time(t):
    return exp(-alpha*t)

# We add the column ri to the dataframe countries
def reproduction_number_by_country(t):
    return (r0 + (-alpha*countries['hdi']))*time(t)/1000000
    
def number_incubated(t):
    ri = reproduction_number_by_country(t)
    contagious_staying = {}
    contagious_coming = {}
    for i in countries.index:
        contagious_staying[i] = countries.loc[i]['incubation'+str(t-1)]*(1 - 
                                1/(countries.loc[i]['healthy'+str(t-1)]
                        - countries.loc[i]['sick'+str(t-1)]))*countries.loc[i]['vi']
        #contagious coming
        neighbors = return_list_neighbors(i, g)
        sum = 0
        for j in neighbors:
            sum += g[j][i]['weight']*countries.loc[j]['incubation'+str(t-1)]/(countries.loc[j]['healthy'+str(t-1)] -
                                      countries.loc[j]['sick'+str(t-1)])
        contagious_coming[i] = sum
    
    countries['recovered'+str(t)] = (1 - mortality_rate)*countries['sick'+str(t-1)]
    countries['infected'+str(t)] = [((contagious_staying[i] 
                + contagious_coming[i])*(1 + ri[i]) - 
                countries.loc[i]['recovered'+str(t)]) for i in countries.index]
    
    countries['healthy'+str(t)] = [countries.loc[i]['population'] - countries.loc[i]['infected'+str(t)] for i in countries.index]
    countries['sick'+str(t)] = countries['incubation'+str(t - 1)] 
    countries['incubation'+str(t)] = countries['infected'+str(t)] - countries['sick'+str(t)]
    
    
add_edge_weights()
print(g['France']['Germany'])
number_incubated(1)
print('France :', countries.loc['France']['infected1'],  countries.loc['France']['healthy1'])
print('Germany :', countries.loc['Germany']['infected1'],  countries.loc['Germany']['healthy1'])
number_incubated(2)
print('France :', countries.loc['France']['infected2'],  countries.loc['France']['healthy2'])
print('Germany :', countries.loc['Germany']['infected2'],  countries.loc['Germany']['healthy2'])

Number of nodes 151
Number of edges 1851
Albania's neighbors : ['Austria', 'Germany', 'Greece', 'Italy', 'Slovenia', 'Turkey', 'United Kingdom']
{'weight': 0.7147198131868132}


TypeError: cannot do label indexing on <class 'pandas.core.indexes.base.Index'> with these indexers [0] of <class 'int'>

In [ ]:
'''
# create a copy and save
def number_of_incubated(t):
    ri = reproduction_number_by_country(countries['country'], t)
    countries['infected'] = (countries['incubation'] -
                            (countries['incubation']/(countries['healthy']-countries['sick']))
                             *country_info[['country']==countries['country']]['nb_voy']) * (1 + ri)
    neighbors = get_neighbors([countries['country']])
    weights = []
    for n in neighbors:
        weights.append(nodes[['i'] == countries['country'] and nodes[['i'] == n]
    return countries

def number_incubated(nodes, i, t):
    ri = reproduction_number_by_country(countries['country'], t)
    neighbors = get_neighbors(i)

    res = countries[['country'] == i]['incubation'] - countries[['country'] == i]['incubation'] / \
    (countries[['country'] == i]['healthy']-countries[['country'] == i]['sick'])* \
    country_info[['country']==i]['nb_voy'] \
    * (1 + ri)
    sum = 0
    for n in neighbors:
        weight = nodes[nodes[['i']== n and nodes[['j'] == i]]['weight']]
        r = weight * (countries[['country']==i]['incubation'] / countries[['country']==i]['inhabitants']
                      - countries[['country']==n]['sick']) * (1 + ri)
        sum += r
    return res + sum


for t in range(1, 10):
    # we do the action over the df of countries
    countries['incubation'] = number_of_incubated(nodes, i, t)
    countries['inhabitants'] = countries['inhabitants'] - mortality_rate * countries['sick']
    countries['recover'] = countries['sick'] * (1 - mortality_rate)
    # next period the incubated people become sick
    countries['sick'] = countries['incubated']
    countries['infected'] = countries['sick'] + countries['incubation']
    countries['healthy'] = countries['inhabitants'] - countries['infected']'''